In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt 
import base64
from PIL import Image
import io
import math 
from math import sqrt
import requests
import cv2
import urllib.request
%matplotlib inline

global embed
saved_model_path = 'E:\Graduation Project\Datasets\PRE TRAINED MOBILE NET'
embed = hub.KerasLayer(saved_model_path)

## CONVERTING IMAGES TO FEATURE VECTORS (EMBEDDINGS)

In [2]:
class TensorVector(object):

    def __init__(self, FileName=None):
        self.FileName = FileName

    def process(self):

        img = tf.io.read_file(self.FileName)
        img = tf.io.decode_jpeg(img, channels=3)
        img = tf.image.resize_with_pad(img, 224, 224)
        img = tf.image.convert_image_dtype(img,tf.float32)[tf.newaxis, ...]
        features = embed(img)
        feature_set = np.squeeze(features)
        return list(feature_set)

## function to find the cosine similarity between feature vectors

In [3]:
def cosineSim(a1,a2):
    sum = 0
    suma1 = 0
    sumb1 = 0
    for i,j in zip(a1, a2):
        suma1 += i * i
        sumb1 += j*j
        sum += i*j
    cosine_sim = sum / ((sqrt(suma1))*(sqrt(sumb1)))
    return cosine_sim

## Path Of Main Uploaded Image

In [5]:
logoImgPath = "E:/Graduation Project/Datasets/WebScraping Logos/punaa.jpg"

logoImage = cv2.imread(logoImgPath, cv2.IMREAD_ANYCOLOR)  
cv2.imshow("main logo",logoImage)
cv2.waitKey(0)

-1

## getting feature vector of the main logo image

In [8]:
helper = TensorVector(logoImgPath)
vector1 = helper.process()
len(vector1)

1280

## getting feature vector of the retrieved logos images

In [10]:
country = "al"
pageNumber = 1
isLastPage = False

while isLastPage != True:
  URL = "http://localhost/WalidLogosApi?country="+country+"&pagenumber="+str(pageNumber)
  response = requests.get(url = URL)
  decodedResponse = response.json()
  #data
  companies = decodedResponse['companies']
  logos = decodedResponse['logos']
  lastPage = decodedResponse["lastPage"]
  #loop through data (both lists have the same length)
  for i in range(len(companies)): 
    companyName = companies[i]
    logoPath = logos[i]
    # check if the logo Image path is hidden or not by the website (if not it starts with http)
    if logoPath[0]!="h":
        #display default image
        logoPath = "E:/Graduation Project/Datasets/WebScraping Logos/hidden-logo.png"
    #extract features from each retrieved logo image
    helper2 = TensorVector(logoPath)
    vector2 = helper2.process()
    # get the cosine similarity
    cosine_similarity = cosineSim(vector1, vector2)
    #display if similarity is above a certain limit
    retrievedLogoImage = cv2.imread(logoPath, cv2.IMREAD_ANYCOLOR) 
    print("Page: "+str(pageNumber)+" Cosine Similarity of The Main Image and Image: "+ logoPath + " is: "+ str(round(cosine_similarity*100))+"%")
    if cosine_similarity > 0.90:
      print(companyName + " is a match")
      urllib.request.urlretrieve(logoPath, "matchedlogo")
      img = Image.open("matchedlogo")
      img.show()
  
  if lastPage == 0:
    isLastPage = True
  #increment pages
  pageNumber = pageNumber + 1





Page: 1 Cosine Similarity of The Main Image and Image: https://d1yjjnpx0p53s8.cloudfront.net/styles/logo-thumbnail/s3/082022/erzeni_shiyak_albaniya.jpg?SjPNVvGNx2U2AA5r4DNgv6Ho8616LsJq&itok=h-c7auM0 is: 54%
Logo of KF Erzeni Shijak is a match
Page: 1 Cosine Similarity of The Main Image and Image: https://d1yjjnpx0p53s8.cloudfront.net/styles/logo-thumbnail/s3/102021/d.png?0ZvbH8iRQxPV_l8gvPdLgcjatwVpsbVB&itok=cUdS2W6N is: 57%
Logo of Digi Print  is a match
Page: 1 Cosine Similarity of The Main Image and Image: https://d1yjjnpx0p53s8.cloudfront.net/styles/logo-thumbnail/s3/092021/lyuftetari_tirana_albaniya_old_logo.jpg?ZfessvDWdk9RcmHrQbr0CCxTTTxg20Zq&itok=GAljqkuV is: 52%
Logo of Luftëtari Tiranë (1950&#039;s logo) is a match
Page: 1 Cosine Similarity of The Main Image and Image: https://d1yjjnpx0p53s8.cloudfront.net/styles/logo-thumbnail/s3/092021/puna_girokastra_albaniya_old_logo.jpg?8QKDFq0.Em2DJJbvBtd_a6su..v6GqOL&itok=A3S8G1mO is: 96%
Logo of Puna Gjirokastër (1950&#039;s logo) is 

KeyboardInterrupt: 